In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = {'sku': input_.loc[a, 'Sku']}

                headers=UA.get_User_Agent_Requests()
                headers['X-Requested-With'] = 'XMLHttpRequest'

                resp = requests.post('https://www.spectreperformance.com/ajaxcatalog/product/applications',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_div = html.xpath('//table[@id="product-vehicles-table"]/tbody/tr/td/div')
                
                # = = = = = = = = = = = = = = =
                
                vehicle = '\n'.join(sorted(list(set([div.xpath('./p[1]/text()')[0].strip() + ' ' + div.xpath('./p[2]/text()')[0].strip() for div in list_div]))))
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Sku': input_.loc[a, 'Sku'],
                                         'Vehicle': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Sku': input_.loc[a, 'Sku']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Sku'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Sku'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Sku'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：128

[ok] - SPC-2002
[尝试次数：4] - [剩余数量：108] - [当前时间：16:27:14]

[ok] - SPC-2033
[尝试次数：6] - [剩余数量：107] - [当前时间：16:27:14]

[ok] - SPC-3013
[尝试次数：5] - [剩余数量：106] - [当前时间：16:27:14]

[ok] - SPC-2022
[尝试次数：3] - [剩余数量：105] - [当前时间：16:27:15]

[ok] - SPC-1001
[尝试次数：7] - [剩余数量：104] - [当前时间：16:27:15]

[ok] - SPC-2038
[尝试次数：8] - [剩余数量：103] - [当前时间：16:27:15]

[ok] - SPC-1012
[尝试次数：1] - [剩余数量：102] - [当前时间：16:27:15]

[ok] - SPC-1010
[尝试次数：1] - [剩余数量：101] - [当前时间：16:27:16]

[ok] - SPC-1001
[尝试次数：4] - [剩余数量：100] - [当前时间：16:27:16]

[ok] - SPA-2395
[尝试次数：3] - [剩余数量：99] - [当前时间：16:27:16]

[ok] - SPC-2000
[尝试次数：9] - [剩余数量：98] - [当前时间：16:27:16]

[ok] - SPC-2044
[尝试次数：1] - [剩余数量：97] - [当前时间：16:27:16]

[ok] - SPC-2003
[尝试次数：11] - [剩余数量：96] - [当前时间：16:27:17]

[ok] - SPC-2007
[尝试次数：12] - [剩余数量：95] - [当前时间：16:27:17]

[ok] - SPA-2498
[尝试次数：10] - [剩余数量：94] - [当前时间：16:27:17]

[ok] - SPC-2050
[尝试次数：5] - [剩余数量：93] - [当前时间：16:27:18]

[ok] - SPC-2055
[尝试次数：13] - [剩余数量：92] - [当前时间：16:27:18]

[ok] - SPC-2013
[尝试次数：12] 